In [ ]:
import json
import pandas as pd
import solar
from timezonefinder import TimezoneFinder
from solar import SolarBasicModel, FixedAxisModel, SingleAxisModel, DualAxisModel, InverterModel, SolarModel, FixedAxisSolarModel, SingleAxisSolarModel, DualAxisSolarModel

basepath="C:/Users/epohl/Desktop/TUNISIA/Cloned_EVOLVE2_6_2_23/EVOLVE2/EVOLVE/src/evolve/src"

with open(basepath+'/SolarPV_input.json') as json_file:
    test_solar_data = json.load(json_file)

irradiance_parameters=test_solar_data['irradiance_parameters']
solar_parameters=test_solar_data['solar_parameters']
inverter_parameters=test_solar_data['inverter_parameters']

tf = TimezoneFinder()
tzone = tf.timezone_at(lng=solar_parameters['pv_location'][1], lat=solar_parameters['pv_location'][0])

if irradiance_parameters['irradiance_file_path'] != '':
    irradiance_dataframe=pd.read_csv(irradiance_parameters['irradiance_file_path']).fillna(value=0)
    irradiance_dataframe['timestamp']=pd.DatetimeIndex(
        pd.to_datetime(irradiance_dataframe['Date']+' '+irradiance_dataframe['Time'])#,
        #tz=tzone
        )
    irradiance_dataframe.drop(
        ['Date','Time'],
            axis=1
            ,inplace=True)

irradiance_dataframe_fixed=irradiance_dataframe.copy(deep=True)
irradiance_dataframe_single_axis=irradiance_dataframe.copy(deep=True)
irradiance_dataframe_single_axis_vertical=irradiance_dataframe.copy(deep=True)
irradiance_dataframe_dual_axis=irradiance_dataframe.copy(deep=True)
# print('irradiance_parameters:',irradiance_parameters)
# print('solar_parameters:',solar_parameters)
# print('inverter_parameters:',inverter_parameters)
# # print('irradiance_dataframe:',irradiance_dataframe)
# print('solar_basic_model:',solar_basic_model)

In [ ]:
#inverter_model
inverter_model=InverterModel()

#fixed axis
solar_basic_model_fixed=SolarBasicModel(
    longitude=solar_parameters['pv_location'][0],
    latitude=solar_parameters['pv_location'][1],
    kw=solar_parameters['kva'],
    irradiance=irradiance_dataframe_fixed
)

fixed_axis_model=FixedAxisModel(
    surface_tilt=45,
    surface_azimuth=180
    )

fixed_axis_solar_model=FixedAxisSolarModel(
    solar_basic_model=solar_basic_model_fixed,
    axis_model=fixed_axis_model,
    inv_model=inverter_model,
    )

fixed_tracking_data,fixed_output=fixed_axis_solar_model.get_inverter_output()

#single axis
solar_basic_model_single_axis=solar.SolarBasicModel(
    longitude=solar_parameters['pv_location'][0],
    latitude=solar_parameters['pv_location'][1],
    kw=solar_parameters['kva'],
    irradiance=irradiance_dataframe_single_axis
)

single_axis_model=SingleAxisModel(
    axis_tilt=0,
    axis_azimuth=180,
    axis_max_tilt=85
    )

single_axis_solar_model=SingleAxisSolarModel(
    solar_basic_model=solar_basic_model_single_axis,
    axis_model=single_axis_model,
    inv_model=inverter_model
    )

single_tracking_data,single_axis_output=single_axis_solar_model.get_inverter_output()

#single_axis_vertical
solar_basic_model_single_axis_vertical=solar.SolarBasicModel(
    longitude=solar_parameters['pv_location'][0],
    latitude=solar_parameters['pv_location'][1],
    kw=solar_parameters['kva'],
    irradiance=irradiance_dataframe_single_axis_vertical
)
single_axis_vertical_model=SingleAxisModel(
    axis_tilt=45,
    axis_azimuth=180,
    axis_max_tilt=85
    )

single_axis_vertical_solar_model=SingleAxisSolarModel(
    solar_basic_model=solar_basic_model_single_axis_vertical,
    axis_model=single_axis_vertical_model,
    inv_model=inverter_model
    )

single_axis_vertical_tracking_data,single_axis_vertical_output=single_axis_vertical_solar_model.get_inverter_output()

#dual axis
solar_basic_model_dual_axis=solar.SolarBasicModel(
    longitude=solar_parameters['pv_location'][0],
    latitude=solar_parameters['pv_location'][1],
    kw=solar_parameters['kva'],
    irradiance=irradiance_dataframe_dual_axis
)

dual_axis_model=DualAxisModel(
    axis_max_tilt=85
)

dual_axis_solar_model=DualAxisSolarModel(
    solar_basic_model=solar_basic_model_dual_axis,
    axis_model=dual_axis_model,
    inv_model=inverter_model
    )

dual_tracking_data,dual_axis_output=dual_axis_solar_model.get_inverter_output()

In [ ]:
#plot PV output
#plot surface tilt and azimuth

from matplotlib import pyplot as plt


time=fixed_output.index.tolist()

fixed=fixed_output['AC_Output'].tolist()
fixed_surface_tilt=[fixed_tracking_data[0]]*len(time)
fixed_surface_azimuth=[fixed_tracking_data[1]]*len(time)

single=single_axis_output['AC_Output'].tolist()
single_surface_tilt=single_tracking_data['surface_tilt'].to_list()
single_surface_azimuth=single_tracking_data['surface_azimuth'].to_list()
single_tracker_angle=single_tracking_data['tracker_theta'].to_list()

single_vertical=single_axis_vertical_output['AC_Output'].tolist()
single_vertical_surface_tilt=single_axis_vertical_tracking_data['surface_tilt'].to_list()
single_vertical_surface_azimuth=single_axis_vertical_tracking_data['surface_azimuth'].to_list()
single_vertical_tracker_angle=single_axis_vertical_tracking_data['tracker_theta'].to_list()

dual=dual_axis_output['AC_Output'].tolist()
dual_surface_tilt=dual_tracking_data['surface_tilt'].to_list()
dual_surface_azimuth=dual_tracking_data['surface_azimuth'].to_list()

fixed_solar_azimuth=irradiance_dataframe_fixed['azimuth'].to_list()
fixed_solar_zenith=irradiance_dataframe_fixed['apparent_zenith'].to_list()

single_solar_azimuth=irradiance_dataframe_single_axis['azimuth'].to_list()
single_solar_zenith=irradiance_dataframe_single_axis['apparent_zenith'].to_list()

dual_solar_azimuth=irradiance_dataframe_dual_axis['azimuth'].to_list()
dual_solar_zenith=irradiance_dataframe_dual_axis['apparent_zenith'].to_list()

fig, ax=plt.subplots()
ax.plot(time,fixed,label='fixed_axis')
ax.plot(time,single,label='single_axis')
ax.plot(time,single_vertical,label='single_axis_vertical')
ax.plot(time,dual,label='dual_axis')
ax.set_title('AC_Output')
ax.set_xlabel('Time')
ax.set_ylabel('kW')
ax.legend();
plt.grid()
plt.show()

# fig, ax=plt.subplots()
# ax.plot(time,fixed_solar_azimuth,label='solar_azimuth')
# ax.plot(time,fixed_solar_zenith,label='solar_apparent_zenith')
# ax.set_title('solar_position')
# ax.set_xlabel('Time')
# ax.set_ylabel('angle')
# ax.legend();
# plt.grid()
# plt.show()


fig, ax=plt.subplots()
ax.plot(time,fixed_surface_tilt,label='surface_tilt')
ax.plot(time,fixed_surface_azimuth,label='surface_azimuth')
ax.plot(time,fixed_solar_azimuth,label='solar_azimuth')
ax.plot(time,fixed_solar_zenith,label='solar_apparent_zenith')
ax.set_title('fixed_axis_tracking_data')
ax.set_xlabel('Time')
ax.set_ylabel('angle')
# ax.set_ylim(bottom=0,top=360)
ax.legend();
plt.grid()
plt.show()

fig, ax=plt.subplots()
ax.plot(time,single_surface_tilt,label='surface_tilt')
ax.plot(time,single_surface_azimuth,label='surface_azimuth')
ax.plot(time,fixed_solar_azimuth,label='solar_azimuth')
ax.plot(time,fixed_solar_zenith,label='solar_apparent_zenith')
# ax.plot(time,single_tracker_angle,label='tracker_theta')
ax.set_title('single_axis_tracking_data')
ax.set_xlabel('Time')
ax.set_ylabel('angle')
# ax.set_ylim(bottom=0,top=360)
ax.legend();
plt.grid()
plt.show()

fig, ax=plt.subplots()
ax.plot(time,single_vertical_surface_tilt,label='surface_tilt')
ax.plot(time,single_vertical_surface_azimuth,label='surface_azimuth')
ax.plot(time,fixed_solar_azimuth,label='solar_azimuth')
ax.plot(time,fixed_solar_zenith,label='solar_apparent_zenith')
# ax.plot(time,single_tracker_angle,label='tracker_theta')
ax.set_title('single_axis_vertical_tracking_data')
ax.set_xlabel('Time')
ax.set_ylabel('angle')
# ax.set_ylim(bottom=0,top=360)
ax.legend();
plt.grid()
plt.show()

fig, ax=plt.subplots()
ax.plot(time,dual_surface_tilt,label='surface_tilt')
ax.plot(time,dual_surface_azimuth,label='surface_azimuth')
ax.plot(time,fixed_solar_azimuth,label='solar_azimuth')
ax.plot(time,fixed_solar_zenith,label='solar_apparent_zenith')
ax.set_title('dual_axis_tracking_data')
ax.set_xlabel('Time')
ax.set_ylabel('angle')
# ax.set_ylim(bottom=0,top=360)
ax.legend();
plt.grid()
plt.show()